# **Pontificia Universidad Javeriana**

## **Gestión de Proyectos para TI**
## **Red Neuronal**

### **Hecho por:**

Isaac Janica

# 1. Importar Librerías y cargar Dataset

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

priorities = pd.read_csv('priorities.csv')
priorities.dropna(inplace=True)
priorities

,to_do,area,due_date,priority
0,adn emprendedor,project management,02/24/2025,1
1,exam,differential equations,02/27/2025,1
2,watch video,web development,02/25/2025,3
3,pset,differential equations,02/24/2025,1
4,study,linear regression,02/27/2025,2
5,exam,linear regression,03/03/2025,2
6,quiz,differential equations,03/05/2025,2
7,pset,distributed systems,02/28/2025,3
8,sprint 2,web development,02/25/2025,2
9,value proposition,project management,03/03/2025,3


In [29]:
priorities.dtypes

to_do       object
area        object
due_date    object
priority     int64
dtype: object

In [30]:
from datetime import datetime
#priorities['due_date'] = priorities['due_date'].str.strip()

priorities['due_date'] = priorities['due_date'].str.strip()

priorities['due_date'] = pd.to_datetime(priorities['due_date'], format='%m/%d/%Y')

priorities.dtypes

to_do               object
area                object
due_date    datetime64[ns]
priority             int64
dtype: object

In [31]:
today = pd.Timestamp.today()
priorities['days'] = (priorities['due_date']-today).dt.days
priorities

,to_do,area,due_date,priority,days
0,adn emprendedor,project management,2025-02-24,1,0
1,exam,differential equations,2025-02-27,1,3
2,watch video,web development,2025-02-25,3,1
3,pset,differential equations,2025-02-24,1,0
4,study,linear regression,2025-02-27,2,3
5,exam,linear regression,2025-03-03,2,7
6,quiz,differential equations,2025-03-05,2,9
7,pset,distributed systems,2025-02-28,3,4
8,sprint 2,web development,2025-02-25,2,1
9,value proposition,project management,2025-03-03,3,7


# 2. Eliminar valores nulos que nos se van a usar y la transformación de variables categóricas

In [32]:
label_encoder = LabelEncoder()
#penguins = penguins.drop('rowid', axis=1)
priorities['to_do'] = label_encoder.fit_transform(priorities['to_do'])
priorities['area'] = label_encoder.fit_transform(priorities['area'])
priorities['due_date'] = label_encoder.fit_transform(priorities['due_date']) 
priorities['days'] = label_encoder.fit_transform(priorities['days'])
priorities['priority'] = label_encoder.fit_transform(priorities['priority'])
priorities

,to_do,area,due_date,priority,days
0,0,3,0,0,0
1,1,0,2,0,2
2,7,4,1,2,1
3,2,0,0,0,0
4,5,2,2,1,2
5,1,2,4,1,4
6,3,0,5,1,5
7,2,1,3,2,3
8,4,4,1,1,1
9,6,3,4,2,4


# 3. Seleccionar las características para el modelo

In [33]:
features = ['to_do', 'area', 'days']
X = priorities[features].values
#y = priorities['priority'].values
priorities['priority'].values

array([0, 0, 2, 0, 1, 1, 1, 2, 1, 2])

# 4 Normalización y entrenamiento

In [34]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

## 4.2 Veamos como se ve X y también su tamaño

In [35]:
X
X.shape

(10, 3)

In [49]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


## 4.3 Ahora veamos como se va a utilizar y_train y y_test


In [50]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

## 5. Vamos generar la arquitectura de la red

In [51]:
model = Sequential([
    Dense(6, activation='tanh', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(6, activation='tanh', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(3, activation='softmax'),
])


/home/isaac/myenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## 6. Compilamos la red

In [52]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, 
                    verbose=1, callbacks=[early_stopping])



Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2857 - loss: 1.1392 - val_accuracy: 0.5000 - val_loss: 0.8431
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.5714 - loss: 0.9635 - val_accuracy: 0.5000 - val_loss: 0.8409
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.5714 - loss: 0.9795 - val_accuracy: 0.5000 - val_loss: 0.8381
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5714 - loss: 0.9846 - val_accuracy: 0.5000 - val_loss: 0.8357
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.1429 - loss: 1.1952 - val_accuracy: 0.5000 - val_loss: 0.8333
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5714 - loss: 0.9790 - val_accuracy: 0.5000 - val_loss: 0.8303
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.4286 - loss: 1.0463 - val_accuracy: 0.5000 - val_loss: 0.8277
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.5714 - loss: 1.0293 - val_accuracy: 0.5000 - val_loss:

Reporte


NameError: name 'actual_classes' is not defined

In [ ]:
features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
X = penguins[features].values
y = penguins['sex'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train
model = Sequential([
    Dense(6, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(3, activation='relu'),
    Dense(2, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, 
                    verbose=1, callbacks=[early_stopping])





In [ ]:
features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
X = penguins[features].values
y = penguins['sex'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train
model = Sequential([
    Dense(6, activation='sigmoid', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(3, activation='relu'),
    Dense(2, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, 
                    verbose=1, callbacks=[early_stopping])





In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"El Accuracy: {test_accuracy:.2f}")
y_pred = model.predict(X_test)
predicted_classes = np.argmax(y_pred, axis=1)
actual_classes = np.argmax(y_test, axis=1)
print("Muestras")
for i in range(5):
    actual = [actual_classes[i]]
    predicted = [predicted_classes[i]]
    print(f"Actual: {actual}, Predicted: {predicted}")
print("Reporte")
print(classification_report(actual_classes, predicted_classes))
